# Synaptic stimulation protocols tests

In [ ]:
# Enable interactive plots with backend 'notebook'
%matplotlib notebook

# print code version (hash of checked out version)
!git log -1 --format="%H"

# print date and time of script execution
import datetime
print("\nNotebook executed at {} in following directory:".format(datetime.datetime.now()))
%cd ..
%pwd
# %connect_info

## Import optimization module

In [ ]:
# Python standard library
import pickle, pprint
pp = pprint.PrettyPrinter(indent=2)

# Distributed logging
from common import logutils
logutils.install_mp_handler()

# BluePyOpt
import bluepyopt.ephys as ephys

# Custom BluePyOpt modules
from optimize.bpop_cellmodels import StnFullModel, StnReducedModel

import optimize.bpop_protocols_stn as StnProtocols
from optimize.bpop_protocols_stn import BpopProtocolWrapper

import optimize.bpop_features_stn as StnFeatures
import optimize.bpop_parameters_stn as StnParameters

from optimize.bpop_analysis_stn import (
    run_proto_responses, plot_proto_responses, 
    save_proto_responses, load_proto_responses,
    plot_responses
)

# Physiology parameters
from evalmodel.cellpopdata import StnModel
from evalmodel.proto_common import StimProtocol

CLAMP_PLATEAU = StimProtocol.CLAMP_PLATEAU
CLAMP_REBOUND = StimProtocol.CLAMP_REBOUND
MIN_SYN_BURST = StimProtocol.MIN_SYN_BURST
SYN_BACKGROUND_HIGH = StimProtocol.SYN_BACKGROUND_HIGH

In [ ]:
# Adjust verbosity of loggers
logutils.setLogLevel('quiet', ['marasco', 'folding', 'redops', 'bluepyopt.ephys.parameters'])

# Full Model
## Create Protocols

In [ ]:
# Protocols to use for optimisation
opt_stim_protocols = [SYN_BACKGROUND_HIGH]

# Make all protocol data
proto_wrappers = [BpopProtocolWrapper.make(p) for p in opt_stim_protocols]
ephys_protos = [p.ephys_protocol for p in proto_wrappers]

# Collect al frozen mechanisms and parameters required for protocols to work
proto_mechs, proto_params = BpopProtocolWrapper.all_mechs_params(proto_wrappers)

## Run Protocols

In [ ]:
# Run protocols using full model to get responses
full_model = StnFullModel(
                name		= 'StnGillies',
                mechs		= proto_mechs,
                params		= proto_params)

nrnsim = ephys.simulators.NrnSimulator(dt=0.025, cvode_active=False)

# Simulate protocols
full_responses = {}
for e_proto in ephys_protos:
    full_responses[e_proto.name] = e_proto.run(
                                        cell_model		= full_model, 
                                        param_values	= {},
                                        sim				= nrnsim,
                                        isolate			= True)

# Plot results
plot_proto_responses(full_responses)

## Make Features & Targets

In [ ]:
# # Make EFEL feature objects
# proto_ids_feats = StnFeatures.make_opt_features(proto_wrappers)

# # Calculate target values from full model responses
# StnFeatures.calc_feature_targets(proto_ids_feats, full_responses)

# Reduced Model
## Make Protocols

In [ ]:
# Protocols to use for optimisation
opt_stim_protocols = [SYN_BACKGROUND_HIGH]

# Make all protocol data
proto_wrappers = [BpopProtocolWrapper.make(p) for p in opt_stim_protocols]
ephys_protos = [p.ephys_protocol for p in proto_wrappers]

# Collect al frozen mechanisms and parameters required for protocols to work
proto_mechs, proto_params = BpopProtocolWrapper.all_mechs_params(proto_wrappers)

## Run Reduced Model

In [ ]:
# Create reduced model and get parameters
red_model = StnReducedModel(
                name		= 'StnFolded',
                fold_method	= 'marasco',
                num_passes	= 7,
                mechs		= proto_mechs,
                params		= proto_params)

nrnsim = ephys.simulators.NrnSimulator(dt=0.025, cvode_active=False)

# Simulate protocols
red_responses = {
    proto.name : proto.run(
                    cell_model		= red_model, 
                    param_values	= {},
                    sim				= nrnsim,
                    isolate			= True)
    for proto in ephys_protos
}

# Plot results
plot_proto_responses(red_responses)